# Second assignment of RT2 course

## User interface

### What does it do?
It is a node of the package rt2_assignment1. Thanks to the user interface the user can interact with a simulation of a robot in Gazebo environment. The robot can be driven o activate to reach different random point on the plane. Moreover, using the Jupyter Notebook's functionalities and the library matplotlib, we can also produce four different graphics about trajectory, time, velocity and goal reached, during the task. 

### The architecture of the rt2_assignment1 package

In this package there are other three nodes besides the user_interface:
1. Go_to_point
2. state_machine
3. random_position


## How to run the code?

Follow the steps below to run the code:

1. Launch the gazebo simulation and the other three nodes typing in the bash:

  `roslaunch rt2_assignment1 sim.launch` *
  
2. Then run the user_interface on Jupyter Notebooks, typing in the bash:

  `jupyter notebook --allow-root`
  
*this launch file contains also another file **user_interface.py** that allows to start and stop the robot using the bash. It was not deleted just because it does not go into conflict with this node. We recommend not to use this possibility, but the user_interface implemented here below to have a full experience of driving.

Below you can find many different libraries and global variables used in the following code. 

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
import actionlib
import actionlib.msg
from actionlib_msgs.msg import GoalID
import rt2_assignment1.msg
from nav_msgs.msg import Odometry
from rt2_assignment1.srv import Command
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
import ipywidgets as widgets
import time
import numpy as np

In [2]:
from IPython.display import display
action=False
cancelled=0
achieved=0
departure=0
arrival=0
travel_time=0
Time = []
stop = False 
rospy.init_node('user_interface')
ac = actionlib.SimpleActionClient('/reaching_goal', rt2_assignment1.msg.PlanningAction)
ui_client = rospy.ServiceProxy('/user_interface', Command)
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
msg = Twist()

## Start and stop the reaching of random goal behavior

This part of the user_interface uses two buttons, one to start and one to stop, the robot during its random behavior. To obtain this result we use the server service /user_interface, that sends the appropriate command, a string, based on the will of the user to start or stop the robot. The buttons uses the method on_click, in this way whenever the button is pressed we have the execution of the code inside the callback **on_botton_clicked1 (or on_botton clicked2)**.

In [3]:
button1 = widgets.Button(description="Start!")
output1 = widgets.Output()
button2 = widgets.Button(description="Stop!")
output2 = widgets.Output()
display(button1, output1)
display(button2, output2)

#handle changes, the observe method of the widget can be used to register a callback.
def on_button_clicked1(b):
    global action,achieved, travel_time, departure, arrival
    with output1:
        action = True
        departure = time.time()
        ui_client("start")
        stop = False
button1.on_click(on_button_clicked1) #ogni volta che viene cliccato viene eseguita questa callback

def on_button_clicked2(b):
    global action,cancelled
    with output2:
        action = True
        ui_client("stop")
        cancelled=cancelled+1
        stop = True
button2.on_click(on_button_clicked2)


Button(description='Start!', style=ButtonStyle())

Output()

Button(description='Stop!', style=ButtonStyle())

Output()

Here below we subscribe to the **/reaching_goal/result** topic. In this way we can be aware of the result of the action, so if the robot has reached the goal.

In [4]:
def result_callback(result):
    global achieved, travel_time, departure, arrival, Time
    arrival = time.time()
    travel_time = arrival-departure
    Time.append(travel_time)
    achieved=achieved+1  
    if not stop:
        departure = time.time()  
    
jr.subscribe ('/reaching_goal/result',rt2_assignment1.msg.PlanningAction,result_callback)

## Drive the robot using the sliders

Here below the robot stops its random behavior, to do that we need to cancel the goal, and the user can drive it using two sliders: one for the linear velocity and the other one for the angular velocity. Anytime a slider is used the method observe, execute the specific callback to stop the action (if it is necessary) and set the velocity that get from the slider. 

In [5]:
a = widgets.FloatSlider(description='lin_vel x',min=-0.5, max=0.5)
display(a)

b = widgets.FloatSlider(description='ang_vel z', min=-0.5, max=0.5)
display(b)

def on_value_change(change):
    global msg
    stop_action ()
    msg.linear.x = change['new']
    pub.publish(msg)
def on_valueang_change(change):
    global msg
    stop_action ()
    msg.angular.z = change['new']
    pub.publish(msg)

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')

FloatSlider(value=0.0, description='lin_vel x', max=0.5, min=-0.5)

FloatSlider(value=0.0, description='ang_vel z', max=0.5, min=-0.5)

this function is called whenever the we need to switch from the random behavior to the driven one.

In [6]:
def stop_action ():
    global action,cancelled,stop
    if action:
        action = False
        ac.cancel_all_goals();
        cancelled=cancelled+1
        stop = False;

## Fully control of the robot using five buttons 

Here below the robot can be controlled using 5 buttons: Go ahead, Turn left, Turn right, Go back, and Stop. For security reason the author of this project decided to place the stop button slightly distant from the others to avoid accidental stops. To control the robot we fill the field of the Twist message linear.x and angular.z and publish the msg in **cmd_vel** topic.

In [7]:
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
b1 = Button(description='Go ahead',
layout=Layout(width='auto', align="center", grid_area='b1'),
style=ButtonStyle(button_color='silver'))
b2 = Button(description='Turn left',
layout=Layout(width='auto', grid_area='b2'),
style=ButtonStyle(button_color='silver'))
b3 = Button(description='Turn right',
layout=Layout(width='auto', grid_area='b3'),
style=ButtonStyle(button_color='silver'))
b4 = Button(description='Go back',
layout=Layout(width='auto', grid_area='b4'),
style=ButtonStyle(button_color='silver'))
b5 = Button(description='STOP!',
layout=Layout(width='auto', grid_area='b5'),
style=ButtonStyle(button_color='red'))
output1 = widgets.Output()
output2 = widgets.Output()
output3 = widgets.Output()
output4 = widgets.Output()
output5 = widgets.Output()

def on_button_clicked1(b1):
    with output1:
        global msg
        stop_action ()
        msg.linear.x = 0.5
        msg.angular.z = 0
        pub.publish(msg)
b1.on_click(on_button_clicked1)

def on_button_clicked2(b2):
    with output2:
        global msg
        stop_action ()
        msg.angular.z = -0.5
        msg.linear.x = 0
        pub.publish(msg)
b2.on_click(on_button_clicked2)

def on_button_clicked3(b3):
    with output3:
        global msg
        stop_action ()
        msg.angular.z = 0.5
        msg.linear.x = 0
        pub.publish(msg)
b3.on_click(on_button_clicked3)

def on_button_clicked4(b4):
    with output4:
        global msg
        stop_action ()
        msg.angular.z = 0
        msg.linear.x = -0.5
        pub.publish(msg)
b4.on_click(on_button_clicked4)

def on_button_clicked5(b5):
    with output5:
        global msg
        stop_action ()
        msg.angular.z = 0
        msg.linear.x = 0
        pub.publish(msg)
b5.on_click(on_button_clicked5)

GridBox(children=[b1, b2, b3, b4, b5],
layout=Layout(
width='100%',
grid_template_rows='auto auto',
grid_template_columns='20% 20% 20% 20% 20%',
grid_template_areas='''
" . b1 .  . b5"
"b2 b4 b3 . ."
''')
)

GridBox(children=(Button(description='Go ahead', layout=Layout(grid_area='b1', width='auto'), style=ButtonStyl…

## Bar plot of cancelled and achieved goals 

In order to show the graph it is necessary anytime to run the specific cell, otherwise it is not updated. We used two variables to take into account the number of reached goal and the cancelled ones. Whenever the callback result_callback is executed, that is just when the action server set the result as true, the variable **achieved** is increases by one, the specular same thing happens for the **cancelled** variable, which increases its value anytime the goal is deleted.

In [8]:
global cancelled, achieved
x = ['cancelled','achieved']
height = [cancelled, achieved]
fig3, ax3 = plt.subplots()
plt.bar(x, height, width=.5, bottom=None, align='center', color= 'orange')
plt.title ('Goals: cancelled vs achieved')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Goals: cancelled vs achieved')

## Histogram of the travel time: from the start point to the goal

This is a graph that shows the travel time of the robot, that is how reaching a goal takes to the robot. We collected this important data using the time method inside the previous function **result_callback** and the callback of the button start **on_button_clicked1**, which start the timer.
To show the plot click on Run for this specific cell, it will be updated at every click

In [9]:
global travel_time, Time
print (travel_time)

fig4, ax4 = plt.subplots()
ax4.hist(Time, histtype='bar', fill=False)
#ax4.legend(prop={'size': 10})
ax4.set_title('histogram of travel time')

0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'histogram of travel time')

## Position of the robot on plane XY during its task

Here below we have a graph that is live updated. It shows the position of the robot in the plane xy during the task. To show the graph we need to run the cell where we subscribe to **/odom** and the one below that, where we call the **FuncAnimation** method.

In [10]:
fig, ax = plt.subplots()

ax.set_xlim(( -10, 10))
ax.set_ylim((-10, 10))

line, = ax.plot([], [], color='b')
x_data=[]
y_data=[]
ax.set_xlabel('position x') 
ax.set_ylabel('position y')
ax.set_title("Position on plane xy")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Position on plane xy')

In [11]:
def init():
    line.set_data([], [])
    return (line)


def odom_callback(msg):
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)


def animate(i):
    line.set_data(x_data, y_data)
    return (line)


jr.subscribe('/odom', Odometry, odom_callback)


In [12]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

## Line Plot: cmd_vel vs actual velocity

In this last cells we implemented a plot that confronts the linear and angular velocities from two different point of view, the linear and angular in the **/cmd_vel** topic and the actual linear and angular velocity in **/odom** topic.
This graph is updated continuously. Please consider that it needs two subscriptions (in two different cells) to two different topics for two fields each one, for this reason it can be a little bit unreliable. Due to that, sometimes could be necessary to restart the following cells clicking on run for each cell.

In [13]:
start = time.time() #start a timer 
time1 = list(range(0, 10))
fig1, ax1 = plt.subplots()
line1, = ax1.plot([], [], color='k', label='/odom')
line2, = ax1.plot([], [], color='r', label='/cmd_vel')
line3, = ax1.plot([], [], color='b', label='/odom angular')
line4, = ax1.plot([], [], color='y', label='/cmd_vel angular')
ax1.set_xlabel('time (s)') 
ax1.set_ylabel('velocity m/s')
ax1.set_title("cmd_vel vs actual velocity")
plt.legend(loc='lower left')
y_data1=[]
y_data2=[]
y_data3=[]
y_data4=[]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
def init1():
    line1.set_data([], [])
    line2.set_data([],[])
    line3.set_data([], [])
    line4.set_data([],[])
    return [line1, line2, line3, line4]


def odom_callback(msg):
    if len(time1)>10:
        time1.pop(0)
     
    if len(y_data1)>10:
        y_data1.pop(0)
    if len(y_data3)>10:
        y_data3.pop(0)
    y_data1.append(msg.twist.twist.linear.x)
    y_data3.append(msg.twist.twist.angular.x)
    time1.append(time.time()-start)

jr.subscribe('/odom', Odometry, odom_callback)

Removing previous callback, only one redirection possible right now


In [15]:
def cmd_vel_callback(msg):
    
    if len(y_data2)>10:
        y_data2.pop(0)
    if len(y_data4)>10:
        y_data4.pop(0)
    y_data2.append(msg.linear.x)
    y_data4.append(msg.angular.z)

jr.subscribe('/cmd_vel', Twist, cmd_vel_callback)


In [16]:
def animate1(i):
    line1.set_data(time1, y_data1)
    line2.set_data(time1, y_data2)
    line3.set_data(time1, y_data3)
    line4.set_data(time1, y_data4)
    ax1.axis([time1[0]-0.2,time1[9]+0.2,-0.5,0.5])   
    return [line1,line2, line3, line4]
anim = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

## Future improvements

In the future, an important upgrade will be showing also the orientation of the robot in the graph of the position in the plane XY and try to avoid possible conflicts between the many subscriptions, for example in the graph cmd_vel vs actual velocity.